<a href="https://colab.research.google.com/github/Mariyaben/Optimized_CNN/blob/main/DesigningCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
mnist=keras.datasets.mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels)=mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255
test_images=test_images/255

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
train_images[0].shape

(28, 28, 1)

In [ ]:
def build_model(hp):
  model=keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter', min_value=50, max_value=150, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28, 28, 1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=50, max_value=80, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=50, max_value=150, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


  return model





In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [ ]:
tuner_search=RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='output', project_name="MNIST")

In [ ]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9879999756813049

Best val_accuracy So Far: 0.9913333058357239
Total elapsed time: 00h 02m 59s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 66)        1716      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 66)        108966    
                                                                 
 flatten (Flatten)           (None, 26400)             0         
                                                                 
 dense (Dense)               (None, 66)                1742466   
                                                                 
 dense_1 (Dense)             (None, 10)                670       
                                                                 
Total params: 1853818 (7.07 MB)
Trainable params: 1853818 (7.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0138 - accuracy: 0.9956 - val_loss: 0.0481 - val_accuracy: 0.9897
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0108 - accuracy: 0.9966 - val_loss: 0.0478 - val_accuracy: 0.9883
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0085 - accuracy: 0.9971 - val_loss: 0.0563 - val_accuracy: 0.9900
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0045 - accuracy: 0.9987 - val_loss: 0.0526 - val_accuracy: 0.9917
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0075 - accuracy: 0.9980 - val_loss: 0.0553 - val_accuracy: 0.9915
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0054 - accuracy: 0.9984 - val_loss: 0.0734 - val_accuracy: 0.9907
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0069 - accuracy: 0.9983 - val_loss: 0.0729 - val_accuracy

In [ ]:
predictions=model.predict(test_images)

313/313 [==============================] - 1s 2ms/step


In [ ]:
np.argmax(predictions, axis=1)

array([7, 2, 1, ..., 4, 5, 6])